In [133]:
from waybacknews.searchapi import SearchApiClient
from datetime import datetime
import pandas as pd
import requests
from retrying import retry
import requests_cache
from tqdm import tqdm
# import mediacloud.api
from newspaper import Article
# import unicodedata
import concurrent.futures
from utils.utils import split_into_chunks, fetch_snippet, sanitize_snippet, get_snippet_from_wayback_machine
from concurrent.futures import ThreadPoolExecutor

import weaviate
import json
import os

In [134]:
# set up weaviate client
OPENAI_APIKEY = os.environ['OPENAI_API_KEY']
# HuggingFace_APIKEY = os.environ['HuggingFace_API_KEY']
client = weaviate.Client(
    url = "https://semantic-search-cluster-mp9jmm6o.weaviate.network",  # Replace with your endpoint
    auth_client_secret=weaviate.AuthApiKey(api_key="kEzuSpJyK8J7IDLLsmzoBlH8mOtkrfkCjIH6"),  # Replace w/ your Weaviate instance API key
    additional_headers = {
        "X-OpenAI-Api-Key": OPENAI_APIKEY
        # "X-Huggingface-Api-Key": HuggingFace_APIKEY
    }
)

if client.schema.exists("Article"):
    client.schema.delete_class("Article")

In [135]:
#for raw text
# class_obj = {
#     "class": "Article",
#     "vectorizer": "text2vec-huggingface",  # "text2vec-openai"
#     "moduleConfig": {
#         "text2vec-huggingface": {
#             "model": "sentence-transformers/all-MiniLM-L6-v2"
#         #     "options": {
#         #         "waitForModel": True,
#         #         "useGPU": True,
#         #         "useCache": True}
#         },
#         # "text2vec-openai": {},
#         "generative-openai": {}  # Ensure the `generative-openai` module is used for generative queries
#     }
# }

# for pre-vectorized data
class_obj = {
    "class": "Article",
    "vectorizer": "none",
    "moduleConfig": {
        "generative-openai": {}  # Ensure the `generative-openai` module is used for generative queries
    }
}

client.schema.create_class(class_obj)

In [136]:

import mediacloud.api
collection_id = "38379429"
api = SearchApiClient("mediacloud")

SOURCES_PER_PAGE = 100  # the number of sources retrieved per page
mc_directory = mediacloud.api.DirectoryApi('e85cce24da8b73eaa05329d258146c044ef055db')
sources = []
offset = 0   # offset for paging through
while True:
    # grab a page of sources in the collection
    response = mc_directory.source_list(collection_id=collection_id, limit=SOURCES_PER_PAGE, offset=offset)
    # add it to our running list of all the sources in the collection
    sources += response['results']
    # if there is no next page then we're done so bail out
    if response['next'] is None:
        break
    # otherwise setup to fetch the next page of sources
    offset += len(response['results'])
print("Collection has {} sources".format(len(sources)))
all_domains = [s['name'] for s in sources]

# Cleaning up domains
cleaned_domains = [
    domain.replace('https://www.', '')
          .replace('http://www.', '')
          .replace('https://', '')
          .replace('http://', '')
          .replace('#spider', '')
          .replace('/#spider', '')
          .rstrip('/')
    for domain in all_domains
    if domain  # Ensure the domain is not None or empty
]
print(f"Number of sources: {len(cleaned_domains)}")
domains_df = pd.DataFrame(cleaned_domains, columns=['Domain'])
# Save the DataFrame to a CSV file
# domains_df.to_csv('domains.csv', index=False)

domain_chunks = list(split_into_chunks(cleaned_domains, 1000))

# Enable requests cache
requests_cache.install_cache('article_cache', backend='filesystem', expire_after=3600)


# Query parameters
# query_term = '("police shooting" OR "shot by police" OR "police shot" OR "officer-involved shooting" OR "police-involved shooting" OR "police officer shooting" OR "police shot" OR "officer shot")'
query_term = '("police shooting" OR "shot by police" OR "police shot" OR "officer-involved shooting" OR "police-involved shooting" OR "police officer shooting" OR "officer shot" OR "deputy shot" OR "sheriff shot" OR "cop shot" OR "trooper shot" OR "shot by officer" OR "shot by deputy" OR "shot by sheriff" OR "shot by cop" OR "shot by trooper" OR \
    "killed by police" OR "killed by officer" OR "killed by deputy" OR "killed by sheriff" OR "killed by cop" OR "killed by trooper")'

# query_term = 'police AND shot'
start = datetime(2023, 12, 1) #11/6 - 11/15
end = datetime(2023, 12, 15) 
language = "en"

# DataFrame to store combined results
combined_results = pd.DataFrame()
results_list = []

for chunk in domain_chunks:
    domains_str = f"domain:({' OR '.join(chunk)})"
    query = f"{query_term} AND language:{language} AND {domains_str}"
    
    # Perform the search with the current chunk
    articles = []
    for list_of_articles in api.all_articles(query, start, end):
        articles.extend(list_of_articles)
        print(f"Found {len(articles)} articles")
    
    if articles:
        chunk_results = pd.DataFrame(articles)
        results_list.append(chunk_results)
    
# Concatenate all DataFrames in the list
combined_results = pd.concat(results_list, ignore_index=True)
# results = pd.DataFrame(articles).sort_values(by='publication_date', ascending=False)
combined_results.sort_values(by='publication_date', ascending=False, inplace=True)
print(combined_results.shape)
combined_results.drop_duplicates(subset=['title'], keep='first', inplace=True)
print("after dropping duplicates: ", combined_results.shape)
combined_results.to_csv(f'../data_storage/{start}_no_snippet.csv', index=False)


for index, article in tqdm(combined_results.iterrows(), total=combined_results.shape[0]):
    # only use wayback machine
    wayback_url = article['article_url']
    snippet = get_snippet_from_wayback_machine(wayback_url)
    if snippet:
        sanitized_snippet = sanitize_snippet(snippet)
        combined_results.loc[index, 'snippet'] = sanitized_snippet
        
    # or concurrently run mediacloud wayback machine and newspaper3k
    # article_url = article['url']
    # wayback_url = article['article_url']
    # snippet, method_used = fetch_snippet(article_url, wayback_url)
    # if snippet:
    #     sanitized_snippet = sanitize_snippet(snippet)
    #     combined_results.loc[index, 'snippet'] = sanitized_snippet
    #     # print(f"Snippet fetched using {method_used} method for URL: {article_url}")

print(combined_results.shape)
combined_results.dropna(subset=['snippet'], inplace=True)
print("after dropping null snippets: ", combined_results.shape)
combined_results.reset_index(drop=True, inplace=True)
combined_results.to_csv(f'../data_storage/{start}.csv', index=False)
print(f"Data retrieval complete. Results saved to '../data_storage/{start}.csv'.")



Collection has 8913 sources
Number of sources: 8913
Found 415 articles
Found 353 articles
Found 86 articles
Found 23 articles
Found 37 articles
Found 29 articles
Found 9 articles
Found 6 articles
Found 3 articles
(961, 9)
after dropping duplicates:  (495, 9)


100%|██████████| 495/495 [02:24<00:00,  3.44it/s]

(495, 10)
after dropping null snippets:  (495, 10)
Data retrieval complete. Results saved to '../data_storage/2023-12-01 00:00:00.csv'.


In [1]:
# pre-vectorize the snippet to avoid using huggingface API which costs money
from sentence_transformers import SentenceTransformer
# model = SentenceTransformer('all-MiniLM-L6-v2')
model = SentenceTransformer('msmarco-MiniLM-L6-cos-v5')
print("Max Sequence Length:", model.max_seq_length)
model.max_seq_length = 512


/Users/karenwang/.virtualenvs/semantic-search/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
.gitattributes: 100%|██████████| 1.18k/1.18k [00:00<00:00, 2.46MB/s]
1_Pooling/config.json: 100%|██████████| 190/190 [00:00<00:00, 1.19MB/s]
README.md: 100%|██████████| 5.13k/5.13k [00:00<00:00, 22.7MB/s]
config_sentence_transformers.json: 100%|██████████| 122/122 [00:00<00:00, 1.22MB/s]
pytorch_model.bin: 100%|██████████| 90.9M/90.9M [00:12<00:00, 7.40MB/s]
sentence_bert_config.json: 100%|██████████| 53.0/53.0 [00:00<00:00, 474kB/s]
special_tokens_map.json: 100%|██████████| 112/112 [00:00<00:00, 577kB/s]
tokenizer.json: 100%|██████████| 466k/466k [00:00<00:00, 913kB/s]
tokenizer_config.json: 100%|██████████| 430/430 [00:00<00:00, 876kB/s]
vocab.txt: 100%|██████████| 232k/232k [00:00<00:00, 10.6MB/s]
modules.js

Max Sequence Length: 384


In [138]:
from tqdm import tqdm
combined_results['snippet_vector'] = None

for index, row in tqdm(combined_results.iterrows(), total=combined_results.shape[0]):
    snippet = row['snippet']
    snippet_vector = model.encode(snippet).tolist()

    # Store embeddings in DataFrame
    combined_results.at[index, 'snippet_vector'] = snippet_vector

print(combined_results.shape)


100%|██████████| 495/495 [00:29<00:00, 16.64it/s]

(495, 11)


In [139]:
# for raw text
# print(combined_results.shape)
# with client.batch(batch_size=100) as batch:
#     for index, row in (combined_results.iterrows()):
#         properties = {
#             "title": row['title'],
#             "publication_date": row['publication_date'],
#             "snippet": row['snippet']
#         }
#         batch.add_data_object(properties, "Article")
#         print(f"Importing article: {index + 1}")

# for pre-vectorized data
client.batch.configure(batch_size=100)
with client.batch as batch:
    for index, row in (combined_results.iterrows()):
        properties = {
            "title": row['title'],
            "publication_date": row['publication_date'],
            "snippet": row['snippet']
        }
        batch.add_data_object(properties, "Article", vector=row["snippet_vector"])

In [140]:
# graphql (allows to group/merge similar results)
# query_text = "Find articles about police shooting that resulted in death of the victims. The incident is about police officer,deputy, sheriff, trooper, cop who fatally fired shots and killed someone. The victims are dead."
query_text = "Recent police shooting incidents leading to fatalities of the victims. The incidents are about police officer, deputy, sheriff, trooper, cop who fatally fired shots and killed someone."
query_vector = model.encode(query_text).tolist()

get_articles_group = f"""
{{
  Get {{
    Article(
      nearVector: {{
        vector: {query_vector}
      }},
      group: {{
        type: merge,
        force: 0.15
      }},
      limit: 100
    ) {{
      title,
      publication_date,
      snippet,
      _additional {{
        generate(
          singleResult: {{
            prompt: \"\"\"Summarize the article: '{'{{snippet}}'}' in one sentence.\"\"\"
          }}
        ) {{
          singleResult,
          error
        }}
      }}
    }}
  }}
}}
"""


query_result = client.query.raw(get_articles_group)
# save to csv
df = pd.DataFrame(query_result['data']['Get']['Article'])
df.to_csv(f'../data_storage/test_weaviate_result.csv', index=False)

In [ ]:
# without grouping

# nearText = {"concepts": ["police shooting, shot by police, police shot, officer-involved shooting, police-involved shooting, police officer shooting, officer shot, deputy shot, sheriff shot, cop shot, trooper shot, shot by officer, shot by deputy, shot by sheriff, shot by cop, shot by trooper, killed by police, killed by officer, killed by deputy, killed by sheriff, killed by cop, killed by trooper"]}
# nearText = {"concepts": ["a recent fatal police shooting. The incident is about police officer fired shots and killed someone. The victims are dead."]}
# nearText = {"concepts": ["Find articles about police shooting that resulted in death of the victims. The incident is about police officer fired shots and killed someone. The victims are dead."]}

query_text = "Find articles about police shooting that resulted in death of the victims. The incident is about police officer,deputy, sheriff, trooper, cop who fired shots and killed someone. The victims are dead. Example: Elk Grove Village police fatally shoot man they say was wielding knife Elk Grove police officers responded to reports of a man with a knife in the 200 block of Fern Drive, and once at the scene, a man exited a house with a knife, police said. He then confronted police in a nearby yard, and during the exchange, police shot him. Elk Grove police officers fatally shot a man whom they say was armed with a knife Friday night outside a home in the northwest suburb. Jack Murray, 24, was shot by officers about 4 p.m. in the 200 block of Fern Drive, according to police and the Cook County Medical Examiner√¢s Office. Officers were responding to reports of a man with a knife. Once officers arrived, Murray allegedly left a house with a knife and confronted police in a nearby yard, and during the exchange, police shot him. Murray was taken to a hospital where he was pronounced dead, police said. Three police officers were also taken to hospitals for observation. Afterward, police said there was no √¢active public safety risk.√¢ An attorney for Murray√¢s family said in a statement that they weren√¢t allowed back into their home late Friday, and that they had √¢no reason to believe that the anything inside the Murrays√¢ family home would be pertinent or relevant to any crime.√¢ √¢We hope to cooperate with the Elk Grove Village police to determine exactly what happened during this tragic incident,√¢ attorney Antonio Romanucci said. Elk Grove police are conducting a criminal investigation, and the regional Major Case Assistance Team is investigating the shooting."
nearVector = {"vector": model.encode(query_text).tolist()}

response = (
    client.query
    .get("Article", ["title", "publication_date", "snippet"])
    .with_near_vector(nearVector)  # Pass the nearVector_dict to with_near_vector
    # .with_generate(single_prompt="Summarize {snippet} in one sentence.")
    .with_limit(50)
    .do()
)

print(json.dumps(response, indent=4))
# convert response to dataframe and save to csv
df = pd.DataFrame(response['data']['Get']['Article'])
df.to_csv(f'../data_storage/{start}_weaviate.csv', index=False)

------
Not used

In [2]:
collection_id = "38379429"
directory_api = mediacloud.api.DirectoryApi(auth_token='e85cce24da8b73eaa05329d258146c044ef055db')
api = SearchApiClient("mediacloud")

In [17]:
# Function to split the domain list into chunks
def split_into_chunks(domains, chunk_size):
    for i in range(0, len(domains), chunk_size):
        yield domains[i:i + chunk_size]


In [4]:
all_domains = []
# Pagination setup
offset = 0
limit = 100  # seems to have a 100 limit
more_pages = True

while more_pages:
    # Fetch sources with current offset
    sources_response = directory_api.source_list(collection_id=collection_id, limit=limit, offset=offset)
    sources = sources_response.get('results', [])
    domains = [source['homepage'] for source in sources]
    all_domains.extend(domains)
    
    # Update the offset
    offset += limit
    
    # Check if there are more pages to fetch
    more_pages = len(sources) == limit

# Cleaning up domains
cleaned_domains = [
    domain.replace('https://www.', '').replace('http://www.', '').replace('https://','').replace('http://','').rstrip('/')
    for domain in all_domains
    if domain  # Ensure the domain is not None or empty
]
print(f"Number of sources: {len(cleaned_domains)}")

domains_df = pd.DataFrame(cleaned_domains, columns=['Domain'])
# Save the DataFrame to a CSV file
domains_df.to_csv('domains.csv', index=False)

Number of sources: 8913


In [21]:
domain_chunks = list(split_into_chunks(cleaned_domains, 1000))

# sources_response = directory_api.source_list(collection_id=collection_id)
# domains = [source['homepage'] for source in sources_response['results']]
# num_sources = len(sources_response['results'])
# print(f"Number of sources: {num_sources}")

# cleaned_domains = [domain.replace('https://www.', '').replace('http://www.', '').replace('https://','').replace('http://','') for domain in domains]
# cleaned_domains = [url.rstrip('/') for url in cleaned_domains]
# print(cleaned_domains)

# Enable requests cache
requests_cache.install_cache('article_cache', backend='filesystem', expire_after=3600)


# Query parameters
query_term = '("police shooting" OR "shot by police" OR "police shot" OR "officer-involved shooting" OR "police-involved shooting" OR "police officer shooting" OR "police shot" OR "officer shot")'
# query_term = '("police shooting" OR "shot by police" OR "police shot" OR "officer-involved shooting" OR "police-involved shooting" OR "police officer shooting" OR "police shot" OR "officer shot" OR "deputy shot" OR "sheriff shot" OR "cop shot")'
# query_term = 'police AND shot'
start = datetime(2023, 9, 1) #11/6 - 11/15
end = datetime(2023, 10, 1) 
language = "en"

# DataFrame to store combined results
combined_results = pd.DataFrame()
results_list = []

for chunk in domain_chunks:
    domains_str = f"domain:({' OR '.join(chunk)})"
    query = f"{query_term} AND language:{language} AND {domains_str}"
    
    # Perform the search with the current chunk
    articles = []
    for list_of_articles in api.all_articles(query, start, end):
        articles.extend(list_of_articles)
        print(f"all_articles endpoint: {len(articles)} articles")
    
    if articles:
        chunk_results = pd.DataFrame(articles)
        results_list.append(chunk_results)
    

# Concatenate all DataFrames in the list
combined_results = pd.concat(results_list, ignore_index=True)
combined_results.sort_values(by='publication_date', ascending=False, inplace=True)
print(combined_results.head())


all_articles endpoint: 622 articles
all_articles endpoint: 261 articles
all_articles endpoint: 67 articles
all_articles endpoint: 15 articles
all_articles endpoint: 34 articles
all_articles endpoint: 25 articles
all_articles endpoint: 14 articles
all_articles endpoint: 14 articles
all_articles endpoint: 2 articles
                                                 title publication_date  \
256  Antioch: Man wanted for homicide is shot by po...       2023-10-01   
112  Antioch: Man wanted for homicide is shot by po...       2023-10-01   
328  1 killed in police shooting at Pa. traffic sto...       2023-10-01   
417  St. Louis police shoot man who threatened rela...       2023-10-01   
173  Demonstrators call for justice after deadly po...       2023-10-01   

             capture_time language            domain  \
256  2023-10-03T11:47:16Z       en   mercurynews.com   
112  2023-10-03T13:10:17Z       en  eastbaytimes.com   
328  2023-10-13T01:29:15Z       en      pennlive.com   
417  2023

In [22]:
combined_results.shape

(1054, 9)

In [23]:
def get_snippet_from_newspaper3k(url):
    extracted_article = Article(url)
    extracted_article.download()
    extracted_article.parse()
    return extracted_article.text

def get_snippet_from_wayback_machine(url):
    response = requests.get(url)
    response.raise_for_status()
    snippet = response.json().get('snippet', '')
    # return response.text 
    return snippet

def fetch_snippet(article_url, wayback_url):
    with concurrent.futures.ThreadPoolExecutor(max_workers=2) as executor:
        future_to_method = {
            executor.submit(get_snippet_from_newspaper3k, article_url): 'newspaper3k',
            executor.submit(get_snippet_from_wayback_machine, wayback_url): 'wayback'
        }
        for future in concurrent.futures.as_completed(future_to_method):
            method = future_to_method[future]
            try:
                data = future.result()
                return data, method
            except Exception as exc:
                print(f"{method} method failed with exception: {exc}")


def sanitize_snippet(snippet):
    sanitized_snippet = snippet.replace('\n', ' ').replace('\r', '').strip()
    # Normalize Unicode characters
    # normalized_snippet = unicodedata.normalize('NFKD', sanitized_snippet)
    return sanitized_snippet

In [ ]:
for index, article in tqdm(combined_results.iterrows(), total=combined_results.shape[0]):
    article_url = article['url']
    wayback_url = article['article_url']
    snippet, method_used = fetch_snippet(article_url, wayback_url)
    if snippet:
        sanitized_snippet = sanitize_snippet(snippet)
        combined_results.loc[index, 'snippet'] = sanitized_snippet
        print(f"Snippet fetched using {method_used} method for URL: {article_url}")

In [ ]:
combined_results.to_csv(f'./data_storage/{start}.csv', index=False)
print(f"Data retrieval complete. Results saved to './data_storage/{start}.csv'.")

In [2]:
!pip install feedparser pandas



[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: pip install --upgrade pip
